In [0]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_colwidth', 100)

# Получаем бесплатные прокси

In [0]:
def get_proxy():
    proxy_arr = []
    for i in range(1, 6):
        r = requests.get(f'http://foxtools.ru/Proxy?page={i}')
        soup = BeautifulSoup(r.text)
#         print(soup.find('table', id='theProxyList').find('tbody').find_all('tr')[0].find_all('td')[1].text)
#         print(soup.find('table', id='theProxyList').find('tbody').find_all('tr')[0].find_all('td')[0].text)
        
        try:
            line = soup.find('table', id='theProxyList').find('tbody').find_all('tr')
            for tr in line:
                td = tr.find_all('td')
                ip = td[1].text
                port = td[2].text
                proxy_arr.append([ip, port])
        except:
            pass
            
#     print(proxy_arr)
    return proxy_arr

def get_another_proxy():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    proxies = []
    for proxy in soup.find_all('tr'):
        try:
            proxies.append(
                [proxy.find_all('td')[0].text,
                 proxy.find_all('td')[1].text])
        except:
            pass
    proxies = proxies[:-29]
#     print(proxies)
    return proxies


proxies = get_proxy() + get_another_proxy()


# Получаем ссылки на все курсы


In [0]:
it_directions = ['big data',
                'biotechnologies',
                'cloud computing',
                'computer games',
                'cybersecurity',
                'digital design',
                'digital marketing',
                'distributed registries',
                'internet of things',
                'it products',
                'machine learning',
                'mobile application',
                'quantum technology',
                'radio engineering',
                'robotics',
                'smart manufacturing',
                'system administration']


In [4]:
import time
from tqdm import tqdm
import requests
from itertools import cycle
import time
import re

proxy_pool = cycle(proxies)
data_links = {}
for direction in it_directions:
    #try:
        links = []
        print('Обрабатываем', direction)

        searched_phrase = direction.replace(' ', '%20')

        url = f'https://www.coursera.org/search?query={searched_phrase}&'

        proxy = proxies[0][0] + ':' + proxies[0][1]

        header = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0",
                  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                  "Accept-Language": "en-US,en;q=0.5",
                  "Accept-Encoding": "gzip, deflate",
                  "DNT": "1",
                  "Connection": "close",
                  "Upgrade-Insecure-Requests": "1"}

        response = requests.get(url,
                                headers=header,
                                proxies={'http': proxy})

        soup = BeautifulSoup(response.text)

        page_amm = int(soup.find_all('button', {'id':'pagination_number_box_button'})[-1].text)

        for i in tqdm(range(1, page_amm+1)):
            proxy = next(proxy_pool)

            url = f'https://www.coursera.org/search?query={searched_phrase}&indices%5Bprod_all_products_term_optimization%5D%5Bpage%5D={i}&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'

            response = requests.get(url,
                                    headers=header,
                                    proxies={'http': proxy[0]+':'+proxy[1]})
            soup = BeautifulSoup(response.text)

            content = soup.find_all('a', {'data-click-key':'search.search.click.search_card'})
            for i in range(len(content)):
              ctype=content[i].findChildren('div', {"class",lambda x: x and x.startswith("productTypePill")})
              links += [['https://www.coursera.org' + content[i]['href'], ctype[0].text]]
        data_links[direction] = links
    #except:
    #    pass
#data_links

Обрабатываем big data


100%|██████████| 23/23 [00:15<00:00,  1.67it/s]


Обрабатываем biotechnologies


100%|██████████| 3/3 [00:02<00:00,  1.50it/s]


Обрабатываем cloud computing


...

100%|██████████| 7/7 [00:06<00:00,  1.38it/s]


Обрабатываем smart manufacturing


100%|██████████| 2/2 [00:02<00:00,  1.25s/it]


Обрабатываем system administration


100%|██████████| 8/8 [00:05<00:00,  1.56it/s]


In [0]:
# Сформируем DataFrame, содержащий ссылки
df_links=pd.DataFrame()
for k,v in data_links.items():
  for el in v:
    d={'direction':k,'url':el[0],'program':el[1]}
    df_links=df_links.append(d,ignore_index=True)



In [6]:
# Отобразим количество ссылок по направлению и типу
df_links.groupby(['direction','program']).size()

direction               program                   
big data                COURSE                        176
                        DEGREE                          2
                        MASTERTRACKâ¢ CERTIFICATE      2
                        PROFESSIONAL CERTIFICATE        4
                        SPECIALIZATION                 42
biotechnologies         COURSE                         21
                        SPECIALIZATION                  1
cloud computing         COURSE                        176
                        DEGREE                          4
                        PROFESSIONAL CERTIFICATE        4
                        SPECIALIZATION                 50
computer games          COURSE                        117
                        PROFESSIONAL CERTIFICATE        2
                        SPECIALIZATION                 23
cybersecurity           COURSE                         43
                        PROFESSIONAL CERTIFICATE        1
                     

In [7]:
# Оставим только специализации, сертификацию и курсы
types=['SPECIALIZATION','PROFESSIONAL CERTIFICATE','COURSE']
mask=df_links.program.isin(types)
df_links=df_links[mask]
df_links.groupby(['direction']).size()

direction
big data                  222
biotechnologies            22
cloud computing           230
computer games            142
cybersecurity              58
digital design            210
digital marketing          90
distributed registries      3
internet of things        102
it products               500
machine learning          307
mobile application         97
quantum technology         12
radio engineering          36
robotics                   65
smart manufacturing        14
system administration      76
dtype: int64

In [0]:
df_links=df_links.reset_index(drop=True)
df_links.index+=1

In [98]:
df_links

,direction,program,url
1,big data,SPECIALIZATION,https://www.coursera.org/specializations/big-data
2,big data,SPECIALIZATION,https://www.coursera.org/specializations/big-data-introduccion
3,big data,SPECIALIZATION,https://www.coursera.org/specializations/big-data-engineering
4,big data,SPECIALIZATION,https://www.coursera.org/specializations/gcp-data-machine-learning
5,big data,COURSE,https://www.coursera.org/learn/big-data-introduction
...,...,...,...
2182,system administration,PROFESSIONAL CERTIFICATE,https://www.coursera.org/professional-certificates/cloud-engineering-gcp
2183,system administration,SPECIALIZATION,https://www.coursera.org/specializations/ibm-microservices
2184,system administration,COURSE,https://www.coursera.org/learn/easing-the-burden-of-obesity-diabetes-cvd
2185,system administration,COURSE,https://www.coursera.org/learn/science-forensique-tribunal


# Парсинг всех ссылок

In [0]:
# Функция по обработке ссылок. Обрабатываем курсы и специализации. В качестве контейнера данных - словарь.
def parse(links):
  
  from tqdm import tqdm
  import requests
  from itertools import cycle
  import time
  
  proxy_pool = cycle(proxies)
  proxy = next(proxy_pool)
  data={}
  lang_ind=3
  for direction in links.groupby(['direction']).groups.keys():
    mask=links['direction']==direction
    current=links[mask]
    print(f'\n{direction}')
    for el in tqdm(current[['url','program']].itertuples(index=False),total=current.shape[0]):
        url=el[0]
        program=el[1]
        # подгрузка след. прокси
        proxy = next(proxy_pool)
        # эмуляция клиента
        header = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0",
                  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                  "Accept-Language": "en-US,en;q=0.5",
                  "Accept-Encoding": "gzip, deflate",
                  "DNT": "1",
                  "Connection": "close",
                  "Upgrade-Insecure-Requests": "1"}
        url = url.replace(' ', '%20')
        response = requests.get(url, headers=header, proxies={'http': proxy[0]+':'+proxy[1]})
        soup = BeautifulSoup(response.text)
        try:
            if program=='COURSE': lang_ind=4
            language= soup.find_all('h4', {"class","H4_1k76nzj-o_O-weightBold_uvlhiv-o_O-bold_1byw3y2 m-b-0"})[lang_ind].text
        except:
            language = None
        if language in ['English']:
          # Если это не курс, а специализация
          if program!='COURSE':
              try:
                  title = soup.find_all('h1')[0].text
              except:
                  title = None
              try:
                  marks = soup.find('span', {"data-reactid":"167"}).text.split(" ")[0]
              except:
                  marks = None
              try:
                  star_rating = soup.find('span', {"data-reactid":"165"}).text
              except:
                  star_rating = None
              try:
                  descr = soup.find('div', {"class","content-inner"}).text
              except:
                  descr = None
              data[title]={
                       'descr':descr,
                       'marks':marks,
                       'rating':star_rating,
                       'url':url,
                       'program': program,
                       'direction':direction
                      }
              # Найдем число ссылок на курсы специализации
              textnum=soup.find_all('h3', {"class","H2_1pmnvep-o_O-weightNormal_s9jwp5-o_O-fontHeadline_1uu0gyz max-text-width-xl m-b-1"})[1].text
              nums=int(re.compile("\d").findall(textnum)[0])
              # Ввиду того, что на странице есть скрытые поля, обработаем первые 4 курса
              if nums>4: nums=4
              for j in range(nums):
                try:
                  course_link = 'https://www.coursera.org'+soup.find_all('a', {'data-e2e':'course-link'})[j]['href']
                except:
                  course_link=None
                if course_link:
                    ctitle,cdata=parse_course(course_link,direction)
                    if ctitle not in data.keys():
                      data[ctitle]=cdata
          else:
            ctitle,cdata=parse_course(url,direction)
            if ctitle not in data.keys():
              data[ctitle]=cdata
    #end for
  return data

def parse_course(course_link,direction):
    course_link = course_link.replace(' ', '%20')
    header = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0",
                  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                  "Accept-Language": "en-US,en;q=0.5",
                  "Accept-Encoding": "gzip, deflate",
                  "DNT": "1",
                  "Connection": "close",
                  "Upgrade-Insecure-Requests": "1"}
    proxy = next(proxy_pool)
    resp = requests.get(course_link, headers=header, proxies={'http': proxy[0]+':'+proxy[1]})
    course_soup = BeautifulSoup(resp.text)
    try:
      title=course_soup.find('div', {"data-test":"banner-title-container"}).text
    except:
      title=None
    try:
      descr= course_soup.find('div', {'class':"content-inner"}).text
    except:
      descr=None
    try:
      marks = course_soup.find('div', {"class":"P_gjs17i-o_O-weightNormal_s9jwp5-o_O-fontBody_56f0wi m-r-1s color-white"}).text.split(" ")[0]
    except:
      marks=None
    try:
      star_rating = course_soup.find('span', {"data-test":"number-star-rating"}).text
    except:
      star_rating = None
    d={'descr':descr,
       'marks':marks,
       'rating':star_rating,
       'url':course_link,
       'program': 'COURSE',
       'direction':direction
      }
    return title,d
    


In [114]:
# Обработаем ссылки
courses=parse(df_links)
df_courses = pd.DataFrame.from_dict(courses,orient='index')
df_courses.reset_index(inplace=True)
df_courses.rename(columns={'index':'title'},inplace=True)

  0%|          | 0/222 [00:00<?, ?it/s]


big data


  0%|          | 0/22 [00:00<?, ?it/s]


biotechnologies


  0%|          | 0/230 [00:00<?, ?it/s]

...

  0%|          | 0/14 [00:00<?, ?it/s]


smart manufacturing


  0%|          | 0/76 [00:00<?, ?it/s]


system administration


100%|██████████| 76/76 [02:02<00:00,  1.07s/it]


In [0]:
df_courses.to_csv("courses.csv")

In [118]:
pd.read_csv("courses.csv",index_col=0)

,title,descr,marks,rating,url,program,direction
0,Big Data Specialization,"Drive better business decisions with an overview of how big data is organized, analyzed, and int...","11,948",4.4,https://www.coursera.org/specializations/big-data,SPECIALIZATION,big data
1,Introduction to Big Data,Interested in increasing your knowledge of the Big Data landscape? This course is for those new...,"5,613",4.6,https://www.coursera.org/learn/big-data-introduction,COURSE,it products
2,Big Data Modeling and Management Systems,"Once youâve identified a big data issue to analyze, how do you collect, store and organize you...","2,202",4.4,https://www.coursera.org/learn/big-data-management,COURSE,computer games
3,Big Data Integration and Processing,"At the end of the course, you will be able to:\n\n*Retrieve data from example database and big d...","1,580",4.4,https://www.coursera.org/learn/big-data-integration-processing,COURSE,cloud computing
4,Machine Learning With Big Data,Want to make sense of the volumes of data you have collected? Need to incorporate data-driven d...,"1,419",4.6,https://www.coursera.org/learn/big-data-machine-learning,COURSE,machine learning
...,...,...,...,...,...,...,...
652,Getting Started on Mainframe with z/OS Commands and Panels,Getting Started with the IBM Mainframe with z/OS Commands and Panels is a mostly hands-on in an ...,62,4.8,https://www.coursera.org/learn/z-commands-and-panels,COURSE,system administration
653,Basic System Programming on IBM Z,The foundational knowledge for the position of an IBM z/OS System Programmer and System Administ...,48,4.8,https://www.coursera.org/learn/system-programming,COURSE,system administration
654,Enterprise System Management and Security,"The world runs on computers. Your watch, your TV, your car. You might be familiar on how to op...",254,4.6,https://www.coursera.org/learn/enterprise-system-management-security,COURSE,system administration
655,Performance Assessment in the Virtual Classroom,Effective measures of performance and assessment are a crucial component of any virtual program....,82,4.2,https://www.coursera.org/learn/performance-assessment,COURSE,system administration


In [119]:
!zip courses.zip courses.csv

  adding: courses.csv (deflated 70%)
